In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import scipy as sp
from datetime import datetime

pd.options.display.max_columns = None

## Data to use

In [2]:
df = pd.read_parquet("../data/data_processed.parquet")

print(df.shape)
df.head()

(116918, 70)


,tcuartosvi,thogar,tcuartos,tdormir,tsanitar,cuandi,cuanhoras,tpersona,buscando,puntaje,tiene_paredes_ladrillo,tiene_paredes_tapia,tiene_paredes_tabla,tiene_piso_cemento,tiene_piso_baldosa,tiene_piso_madera_tierra,es_estrato_1y2,tiene_inodoro_alcantarillado,tiene_letrina,tiene_agua_acueducto,cocinan_electricidad,cocinan_gas_natural,cocinan_gas_propano,tiene_eps_subsidiada,tiene_eps_contributiva,tiene_eps_ninguna,tiene_nivel_educativo_ninguna,tiene_nivel_educativo_primaria,tiene_nivel_educativo_secundaria,es_zona_urbana,tiene_servicio_telefono,es_vivi_casa_apto,tiene_energia,tiene_alcanta,tiene_gas,tiene_servicio_basura,tiene_acueducto,es_vivi_arriendo,es_vivi_prop_pagando,es_vivi_prop,es_vivi_otro,tiene_sanitario_dentro_vivi,tiene_sanitario_exclusivo,tiene_ducha,tiene_agua_7dias,tiene_agua_24h,tiene_espacio_cocinar,tiene_cocina_exclusiva_vivi,tiene_telefono_exclusivo_vivi,tiene_nevera,tiene_lavadora,tiene_televisor,tiene_televisor_parabolica,tiene_calentador,tiene_horno,tiene_aire,tiene_computador,tiene_equipo,tiene_moto,tiene_tractor,tiene_auto,tiene_bieraices,sexo_masculino,embarazada_o_con_hijos,asiste_centro_educativo,tiene_ingresos,es_extranjero,paga_arriendo,trabaja_servicios_domestico,ingresos
formulario,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
296071,4.0,1.0,4.0,2.0,1.0,0.0,0.0,2.0,0.0,38.89,0,1,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,500000.0
2000000,4.0,1.0,4.0,2.0,1.0,0.0,0.0,3.0,0.0,57.27,1,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,0,1,0,1,1,1,0,0,0,1280000.0
2000001,4.0,1.0,4.0,3.0,2.0,0.0,0.0,4.0,0.0,68.32,1,0,0,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,0,0,0,800000.0
2000002,2.0,1.0,2.0,2.0,1.0,0.0,0.0,5.0,0.0,36.30,1,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1057000.0
2000003,6.0,1.0,6.0,5.0,1.0,0.0,0.0,8.0,0.0,52.05,1,0,0,0,1,0,1,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,1294000.0


In this case, I'm going to omit the train-test split, the idea is just to ilustrate the Boruta algorithm.

In [3]:
y_name = "puntaje"
X = df.drop(columns=[y_name])
y = df[y_name].copy()

## Boruta Algorithm from scratch

1. Building shadow features: remember that features do not compete among themselves. Instead they compete with a randomized version of them.

In [4]:
seed = 421

In [5]:
# make X_shadow by randomly permuting each column of X
np.random.seed(seed)
X_shadow = X.apply(np.random.permutation)
X_shadow.columns = ['shadow_' + feat for feat in X.columns]

# make X_boruta by appending X_shadow to X
X_boruta = pd.concat([X, X_shadow], axis = 1)

In [11]:
start = datetime.now()

# fit a random forest (suggested max_depth between 3 and 7)
rf = RandomForestRegressor(max_depth = 4, random_state = seed)
rf.fit(X_boruta, y)

# store feature importances
feat_imp_X = rf.feature_importances_[:len(X.columns)]
feat_imp_shadow = rf.feature_importances_[len(X.columns):]

finish = datetime.now()
print("Time execution:", finish - start)

Time execution: 0:02:12.096295


In [12]:
# compute hits
max_imp_shadow = feat_imp_shadow.max()
print("Max importance shadow:", max_imp_shadow)

hits = feat_imp_X > max_imp_shadow
print("Num features hits:", hits.sum())

Max importance shadow: 0.0
Num features hits: 15


In [13]:
# result
pd.DataFrame({"features": X.columns,
              "importance": feat_imp_X,
              "hits": hits.astype(int)}).head()

,features,importance,hits
0,tcuartosvi,0.000000,0
1,thogar,0.000573,1
2,tcuartos,0.000000,0
3,tdormir,0.000000,0
4,tsanitar,0.000000,0


2. Iterations: Binomial distribution

In [14]:
# We can put the before process in a function
def compute_hits_rf(X, seed):

    # make X_boruta
    np.random.seed(seed)
    X_shadow = X.apply(np.random.permutation)
    X_shadow.columns = ['shadow_' + feat for feat in X.columns]
    X_boruta = pd.concat([X, X_shadow], axis = 1)
    
    # fit a random forest 
    rf = RandomForestRegressor(max_depth = 4, random_state = seed)
    rf.fit(X_boruta, y)

    # store feature importances
    feat_imp_X = rf.feature_importances_[:len(X.columns)]
    feat_imp_shadow = rf.feature_importances_[len(X.columns):]
    
    # compute hits
    max_imp_shadow = feat_imp_shadow.max()
    hits = feat_imp_X > max_imp_shadow
    
    return hits

In [15]:
start = datetime.now()

n_iter = 20
df_results = pd.DataFrame({"features": X.columns})

for iter_ in range(n_iter):
    
    print("="*10, "iteration:", iter_, "="*10)
    hits_iter_ = compute_hits_rf(X, seed=iter_)
    df_results["hits_" + str(iter_)] = hits_iter_
    
df_results = df_results.set_index(["features"])
df_results["total_hits"] = df_results.sum(axis=1)

finish = datetime.now()
print("Time execution:", finish - start)

========== iteration: 0 ==========
========== iteration: 1 ==========
========== iteration: 2 ==========
========== iteration: 3 ==========
========== iteration: 4 ==========
========== iteration: 5 ==========
========== iteration: 6 ==========
========== iteration: 7 ==========
========== iteration: 8 ==========
========== iteration: 9 ==========
========== iteration: 10 ==========
========== iteration: 11 ==========
========== iteration: 12 ==========
========== iteration: 13 ==========
========== iteration: 14 ==========
========== iteration: 15 ==========
========== iteration: 16 ==========
========== iteration: 17 ==========
========== iteration: 18 ==========
========== iteration: 19 ==========
Time execution: 0:14:14.478653


In [16]:
df_results.head()

,hits_0,hits_1,hits_2,hits_3,hits_4,hits_5,hits_6,hits_7,hits_8,hits_9,hits_10,hits_11,hits_12,hits_13,hits_14,hits_15,hits_16,hits_17,hits_18,hits_19,total_hits
features,,,,,,,,,,,,,,,,,,,,,
tcuartosvi,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,1
thogar,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,20
tcuartos,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
tdormir,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
tsanitar,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0


In [17]:
alpha = 0.1
thr_rej_area = sp.stats.binom.ppf(alpha / 2, n_iter, .5)
thr_conf_area = sp.stats.binom.ppf(1 - (alpha / 2), n_iter, .5)

In [18]:
print("="*40)
print(f"n_iter = {n_iter}")
print(f"rejected area <= {thr_rej_area} hits")
print(f"tentative area (to check) {thr_rej_area} < hits < {thr_conf_area}")
print(f"confirmed area >= {thr_conf_area} hits")

n_iter = 20
rejected area <= 6.0 hits
tentative area (to check) 6.0 < hits < 14.0
confirmed area >= 14.0 hits


In [19]:
cond = [df_results["total_hits"] <= thr_rej_area,
        df_results["total_hits"] >= thr_conf_area]
res = ["rejected", "confirmed"]

df_results["area"] = np.select(cond, res, default="tentative")

In [20]:
df_results["area"].value_counts()

rejected     54
confirmed    15
Name: area, dtype: int64

In [21]:
df_results.query("area == 'confirmed'").index

Index(['thogar', 'tiene_paredes_ladrillo', 'tiene_inodoro_alcantarillado',
       'tiene_agua_acueducto', 'tiene_eps_subsidiada',
       'tiene_nivel_educativo_ninguna', 'tiene_nivel_educativo_primaria',
       'es_zona_urbana', 'es_vivi_casa_apto', 'tiene_alcanta',
       'tiene_sanitario_exclusivo', 'tiene_ducha', 'tiene_agua_7dias',
       'tiene_agua_24h', 'tiene_lavadora'],
      dtype='object', name='features')

---